In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import lxml
from bs4 import BeautifulSoup

<H2> Функция для запроса данных с бесконечно пролистываемой страницы

In [2]:
def get_Comics_List(page: int):
    json ={
        "sort":"rate",
        "dir":"desc",
        "page":page,
        "types":["9", "8"],
        "site_id":"1",
        "type":"manga",
        "caution_list":["Отсутствует", "16+", "18+"]
    } # json для полечения комиксов западных и русских
    headers ={
        "X-Csrf-Token":None,
        "X-Requested-With":"XMLHttpRequest",
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0"
    }
    url = "https://mangalib.me/api/list"

    session = requests.session()

    html = session.get("https://mangalib.me") #Получаем базовую страницу
    soup = BeautifulSoup(html.text, 'lxml')
    csrf_token = soup.find("meta", {"name":"_token"}).get("content")
    headers["X-Csrf-Token"] = csrf_token #Запоминаем токен

    session.headers.update(headers)
    manga_list = session.post(url, json=json) #Получаем данные
    return manga_list.json()

<h4> Считываем первые 1200 записей

In [3]:
manga_mas = []
for i in range(200):
    manga_mas.append(get_Comics_List(i+1))
manga_mas

AttributeError: 'NoneType' object has no attribute 'get'

Смотрим данные, которые мы вытянули из запросов

In [ ]:
manga_mas[0]['items']['data'][0].keys()

<h2> Заполняем DF

In [ ]:
from random import random
from time import sleep
href_mas = []
data_mas = []
name_mas = []
df = pd.DataFrame()
i = 0
for manga_mas_lower in manga_mas:
    data = manga_mas_lower['items']['data']
    if not data[1] in name_mas:
        name_mas.append(data[1])
        for elems in data:
            for key in list(elems.keys()):
                # print(elems[key])
                if key != 'covers':
                    df.loc[i, key] = elems[key]
                else:
                    for kkey in elems[key].keys():
                        df.loc[i, "cover_"+kkey] = elems[key][kkey]
            # df.loc[i,'href'] = elems['href']
            # df.loc[i,'id'] = int(elems['id'])
            i+=1
df.head()


Функция для красивого статус бара

In [ ]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )


In [ ]:
stop = 0

<h2>Читаем данные со страниц комиксов

In [ ]:
import sys

aim_heads = {'Год релиза':'Release_Year', 'Статус тайтла':'Title_Status', 'Статус перевода':'Translate_Status', 
             'Автор':'Author', 'Художник':'Artist',  'Возрастной рейтинг':'Age_restriction', 'Загружено глав':'Chapters_Loaded'}

for href in enumerate(log_progress(df['href'], every = 1)):
    if href[0] < stop:
        continue
    heads = []
    bodies = {}
    response = requests.get(href[1])
    bs = BeautifulSoup(response.text)
    # Колонка с базовыми данными о комиксе
    heads_bs = bs.find_all('div', {"class":"media-info-list__title"})
    bodies_bs = bs.find_all('div', {"class":"media-info-list__value"})
    
    for i in heads_bs:
        heads.append(i.text)
    for i in enumerate(heads):
        if i[1] in aim_heads.keys() :
            # print(aim_heads[i[1]])
            bodies[aim_heads[i[1]]] = bodies_bs[i[0]].text.replace('\n','') #''.join(filter(str.isalnum, bodies_bs[i[0]].text))
    # print(heads)
    # print(bodies)
    for i in bodies.keys():
        df.loc[href[0], i] = bodies[i]

    # Краткое описание
    desc = bs.find('div', {'class':'media-description__text'})
    if desc:
        df.loc[href[0], 'Short_Description'] = desc.text.replace('\n',' ').replace('\r',' ')
        # print(df.loc[href[0], 'Short_Description'])
    
    # Множество тэгов
    tags = bs.find('div', {'class':'media-tags'})
    tag_mas = []
    if tags != None:
        for i in tags:
            txt = i.text.replace('\n','')
            if txt != '':
                tag_mas.append(txt)
    df.loc[href[0], 'tags'] = str(tag_mas)
    
    # Списки пользователей и оценки
    user_lists = {}
    user_rates = {}
    media_cols = bs.find_all('div', {'class':'media-section__col'})
    for col in media_cols:
        col_type = col.find('div', {'class':'media-section__title'}).text
        if col_type != "Оценки пользователей":
            divs = col.find_all('div',{'class':'media-stats-item__column media-stats-item__count'})
            names = col.find_all('div',{'class':'media-stats-item__column media-stats-item__title'})
            for i in range(len(divs)):
                user_lists[names[i].text.replace('\n','')] = int(divs[i].text)
        else:
            divs = col.find_all('div',{'class':'media-stats-item__column media-stats-item__count'})
            names = col.find_all('div',{'class':'media-stats-item__column media-stats-item__title'})
            for i in range(len(divs)):
                user_rates[int(names[i].text.replace('\n',''))] = int(divs[i].text)
    df.loc[href[0], 'User_rates'] = str(user_rates)
    df.loc[href[0], 'User_lists'] = str(user_lists)

    # Списки переводчиков
    trans_dict = {}
    trans = bs.find('div', {'class':'team-list'})
    if trans != None:
        tr_hrefs = trans.find_all('a', {'class':'team-list-item team-list-item_xs'})
        tr_names = trans.find_all('div', {'class':'team-list-item__name'})
        for i in range(len(tr_names)):
            trans_dict[tr_names[i].text] = tr_hrefs[i].get('href')
    df.loc[href[0], 'Translators'] = str(trans_dict)
    stop += 1
df.head()


"Потрачено на открытие страницы:  0.6658093929290771  
Потрачено на заполнение df:  0.02171039581298828"  
Судя по всему, больше всего времени тратится на открытие каждой новой страницы

Описание итогового DF

In [ ]:
df.info()

title_status_id - бесполезный параметр, его следует убрать

In [ ]:
df.drop('title_status_id', axis=1)
df.to_csv('comics.csv')
df.head()

Итого: 28 различных параметров на 1200 объектов
В ходе выполнения данной лабораторной работы я познакомился с библиотекой BeautifulSoup и научился добывать данные из веб-страниц  
Полученные данные в дальнейшем могут быть использованы для комплексного сравнения различных комиксов и их жанров по популярности